# Programowanie liniowe (Badania operacyjne) - Algorytm transportowy w Python

##### Import wymaganych bibliotek
- pulp - biblioteka Pythona służącą do rozwiązywania problemów optymalizacyjnych w programowaniu linowym

In [6]:
# Import biblioteki pulp
from pulp import *

# Utworzenie programu liniowego
prob = LpProblem("Algorytm transportowy", LpMinimize)

##### Zdefiniowanie problemu
*Firma produkcyjna ma trzy fabryki (F1, F2, F3), które produkują różne produkty. Produkty te muszą być dostarczone do trzech magazynów (M1, M2, M3), z których zostaną przetransportowane do klientów. <br>
Poniżej znajduje się lista fabryk, magazynów oraz koszty transportu między nimi:*

*Koszty transportu:*

|    | M1 | M2 | M3 |
|----|----|----|----|
| F1 | 10 | 20 | 15 |   
| F2 | 18 | 12 | 20 |   
| F3 | 25 | 24 | 10 |   

*Dodatkowe informacje:*

*Dostępność jednostkowa produktów w fabrykach: F1 (60), F2 (40), F3 (90)
Zapotrzebowanie w magazynach: M1 (50), M2 (60), M3 (80)*

*Celem jest opracowanie planu transportu, który spełnia powyższe ograniczenia dostępności i popytu oraz minimalizuje łączny koszt transportu.*

In [7]:
# Zdefiniowanie dostawców
warehouses = ["F1", "F2", "F3"]

# Zdefiniowanie podaży
supply = {"F1": 60, "F2": 40, "F3":90}

# Zdefiniowanie odbiorców
projects = ["M1", "M2", "M3"]

# Zdefiniowanie popytu
demand = {
    "M1": 50,
    "M2": 60,
    "M3": 80,
}

# Utworzenie macierzy kosztów
costs = [ 
    [10,20,15],   
    [18,12,20],
    [25,24,10]  
]

costs = makeDict([warehouses, projects], costs, 0)

In [8]:
# Utworzenie dróg pomiędzy dostawcami a odbiorcami
Routes = [(w, b) for w in warehouses for b in projects]

vars = LpVariable.dicts("Droga", (warehouses, projects), 0, None, LpInteger)

In [9]:
# Dodanie funkcji kosztu do programu liniowego
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Koszt",
)

# Dodanie ograniczeń do programu liniowego
for w in warehouses:
    prob += (
        lpSum([vars[w][b] for b in projects]) <= supply[w],
        "Sum_of_Products_out_of_warehouses_%s" % w,
    )

for b in projects:
    prob += (
        lpSum([vars[w][b] for w in warehouses]) >= demand[b],
        "Sum_of_Products_into_projects%s" % b,
    )

##### Wyznaczenie optymalnego rozwiązania

In [10]:
prob.solve()

# Wypisanie dróg
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# Wypisanie wartości funkcji kosztu
print("Zminimalizowany koszt = ", value(prob.objective))

Droga_F1_M1 = 50.0
Droga_F1_M2 = 10.0
Droga_F1_M3 = 0.0
Droga_F2_M1 = 0.0
Droga_F2_M2 = 40.0
Droga_F2_M3 = 0.0
Droga_F3_M1 = 0.0
Droga_F3_M2 = 10.0
Droga_F3_M3 = 80.0
Zminimalizowany koszt =  2220.0
